# Preparacion inicial

In [12]:
%pip install -q gostcrypto

In [13]:
import hashlib, gostcrypto, base64

# Hashes

In [15]:
def sha256(texto = "hello") -> str:
        return hashlib.sha256(texto.encode(encoding="utf-8")).hexdigest()

print(sha256())
print(sha256("Hola"))

2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824
e633f4fc79badea1dc5db970cf397c8248bac47cc3acf9915ba60b5d76b0e88f


In [8]:
def sha512(texto = "hello") -> str:
        return hashlib.sha512(texto.encode(encoding="utf-8")).hexdigest()

print(sha512())
print(sha512("Hola"))

9b71d224bd62f3785d96d46ad3ea3d73319bfbc2890caadae2dff72519673ca72323c3d99ba5c11d7c7acc6e14b8c5da0c4663475c2e5c3adef46f73bcdec043
c096860be238d7e0a6d3929c7ba06f468d3e6b7b28132ba48d553f845788c513004b1ec78758f24e9e1f006ae9a89651f80023f5505927a7aecd6529fa12c081


In [20]:
def gost(texto = "hello", modo: int = 0) -> str:
    modos = ('streebog256', 'streebog512')
    conversion_bytes = texto.encode()
    gost_obj = gostcrypto.gosthash.new(modos[modo], data=conversion_bytes)
    gost_hash = gost_obj.hexdigest()
    return gost_hash

print(gost("Hola"))
print(gost("Hola", 1))

0751a3486f19e8b9e55b911f582b3520512606ba2b9b8c35f1d74f74b747a118
a7487b537aeee10fe67e7fad8f11eb193fa975368628599484dcac90bbd30544cbb7e624cb9077aab2257aa3fc4cf86403d6830aff2819238920183da1590f35


# Encriptaciones

In [11]:
def busqueda(abecedario: tuple, letra: str):
        posicion = -1
        try:
            posicion = abecedario.index(letra)
        except:
            pass
            
        return posicion
        

def ROT13(cadena: str) -> str:
    cifrado = ""
    conversion = cadena.lower()
    abecedario_1 = ("a","b","c","d","e","f","g","h","i","j","k","l","m")
    abecedario_2 = ("n","o","p","q","r","s","t","u","v","w","x","y","z")
    for caracter in conversion:
        posicion_abecedario_1 = busqueda(abecedario_1, caracter)
        posicion_abecedario_2 = busqueda(abecedario_2, caracter)
        if posicion_abecedario_1 != -1:
            cifrado += abecedario_2[posicion_abecedario_1]
        elif posicion_abecedario_2 != -1:
            cifrado += abecedario_1[posicion_abecedario_2]

    return cifrado

print(ROT13("Hola"))

ubyn


In [39]:
def busqueda(abecedario: tuple, letra: str):
        posicion = -1
        try:
            posicion = abecedario.index(letra)
        except:
            pass
            
        return posicion
    
def cifrado_atbash(texto: str = "hello"):
    abecedario = ["a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v","w","x","y","z"]
    abecedario_invertido = list(reversed(abecedario))
    
    cifrado = ""
    conversion = texto.lower()
    
    for caracter in conversion:
        posicion_abecedario = busqueda(abecedario, caracter)
        if posicion_abecedario != -1:
            cifrado += abecedario_invertido[posicion_abecedario]
    
    return cifrado

print(cifrado_atbash())
print(cifrado_atbash("Hola"))

svool
sloz


In [6]:
def cifrado_hill(texto: str, matriz: list):
    # https://es.wikipedia.org/wiki/Cifrado_Hill
    # https://www.youtube.com/watch?v=ChOhsL-zvBo
    abecedario = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 
                  'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 
                  'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 
                  'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25, 'None': 26}

    texto_mayusculas = texto.upper()

    # Pasamos las letras a numeros
    codificacion = list()
    for caracter in texto_mayusculas:
        codificacion.append(abecedario[caracter])

    # Nos aseguramos que el numero de longitud de la cadena sea multiplo del tamaño de la matriz
    while len(codificacion) % len(matriz[0]) != 0:
        codificacion.append(26)

    # Dividir la lista del mensaje (codificacion) en bloques, tal que la longitud de cada bloque sea igual al ancho de la fila de la matriz cuadrada
    ancho_matriz = len(matriz[0])
    codificacion = [codificacion[i:i + ancho_matriz] for i in range(0, len(codificacion), ancho_matriz)]

    # Realizamos el calculo de matrices. A Bloques de codificacion y B seria la matriz cuadrada.
    a_rows, a_cols = len(codificacion), len(codificacion[0])
    b_rows, b_cols = len(matriz), len(matriz[0])
    c = [[0 for _ in range(b_cols)] for _ in range(a_rows)]
      
    for i in range(a_rows):
        for j in range(b_cols):
            for k in range(a_cols):
                c[i][j] += codificacion[i][k] * matriz[k][j]
        
    # Convertirmos la matriz c en el texto cifrado
    return "".join(chr(numero) for sublist in c for numero in sublist)

def descifrado_hill(texto_cifrado: str, matriz: list):
        
    def calcular_determinante(matriz):
        return matriz[0][0] * matriz[1][1] - matriz[0][1] * matriz[1][0]

    def calcular_matriz_inversa(matriz):
        det = calcular_determinante(matriz)
        if det == 0:
            raise ValueError("La matriz no es invertible (determinante igual a 0)")

        inversa = [[matriz[1][1] / det, -matriz[0][1] / det],
                  [-matriz[1][0] / det, matriz[0][0] / det]]
        return inversa
        
    caracteres_cifrado = list(texto_cifrado)
    cifrado = [ord(caracter) for caracter in caracteres_cifrado]
        
    ancho_matriz = len(matriz[0])
    codificacion = [cifrado[i:i + ancho_matriz] for i in range(0, len(cifrado), ancho_matriz)]
        
    matriz_inversa = calcular_matriz_inversa(matriz)
        
    mensaje_original = []
    for fila in codificacion:
        fila_original = [round(fila[0] * matriz_inversa[0][0] + fila[1] * matriz_inversa[1][0]),
                         round(fila[0] * matriz_inversa[0][1] + fila[1] * matriz_inversa[1][1])]
            
        if 26 in fila_original:
            fila_original.remove(26)

        mensaje_original.extend(fila_original)

    abecedario = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    mensaje_descifrado = ''.join(abecedario[numero % 26] for numero in mensaje_original)
    return mensaje_descifrado

matriz_cuadrada_2_2 = [[12, 23], [14, 7]] # Siempre numeros menores a 26
matriz_cuadrada_2_2 = [[21, 3], [5, 7]] # Siempre numeros menores a 26

texto_cifrado = cifrado_hill("Hola", matriz_cuadrada_2_2)
print(f"Cifrado de la cadena Buscar -", texto_cifrado)
texto_descifrado = descifrado_hill(texto_cifrado, matriz_cuadrada_2_2)
print(f"Descifrado de la cadena Buscar -", texto_descifrado)

Cifrado de la cadena Buscar - Ùwç!
Descifrado de la cadena Buscar - HOLA


# Codificaciones

In [9]:
def codificar_base64(texto = "hello") -> bytes:
    codificacion = texto.encode()
    paso_bytes = bytes(codificacion)
    return base64.b64encode(paso_bytes)

print(codificar_base64())
print(codificar_base64("Hola"))

b'aGVsbG8='
b'SG9sYQ=='


In [10]:
def codificar_base64(texto = "hello") -> bytes:
    codificacion = texto.encode()
    paso_bytes = bytes(codificacion)
    return base64.b32encode(paso_bytes)

print(codificar_base64())
print(codificar_base64("Hola"))

b'NBSWY3DP'
b'JBXWYYI='


In [14]:
def codificar_base16(texto = "hello") -> bytes:
        """
        Con esta funcion codificas 
        el texto en base hexadecimal
        """
        codificacion = texto.encode()
        paso_bytes = bytes(codificacion)
        return base64.b16encode(paso_bytes)
    
print(codificar_base16())
print(codificar_base16("Hola"))

b'68656C6C6F'
b'486F6C61'


In [15]:
def codificar_decimal(texto = "hello") -> list:
    return [ord(caracter) for caracter in texto]

print(codificar_decimal())
print(codificar_decimal("Hola"))

[104, 101, 108, 108, 111]
[72, 111, 108, 97]
